<a href="https://colab.research.google.com/github/gmestrallet/BasicScRNAseq/blob/main/BasicScRNAseq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook demonstrates how to do basic scRNAseq analysis using scanpy with data stored in google drive.

In [ ]:
#Mount Google Drive to access your files, if they are stored there.
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#Set the path where you want to store the files (use your own directory).
import os

In [ ]:
#Replace 'RNAseq_folder' with the path to the folder in your Google Drive or use '/content/' for local storage.
rna_seq_path = '/content/drive/My Drive/RNAseq_folder'
os.chdir(rna_seq_path)

In [ ]:
#Create directories for your data and figures
os.makedirs('data', exist_ok=True)  # Creates 'data' directory if it doesn't exist
os.chdir('data')

In [ ]:
#Create 'write' directory inside 'data'
os.makedirs('write', exist_ok=True)

In [ ]:
#Install necessary libraries and import
!pip install scanpy  # Make sure scanpy is installed
!pip install igraph  # Make sure igraph is installed
!pip install leidenalg  # Make sure leidenalg is installed
import numpy as np
import pandas as pd
import scanpy as sc
import matplotlib.pyplot as plt

In [ ]:
sc.settings.verbosity = 3             # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.logging.print_header()
sc.settings.set_figure_params(dpi=80, facecolor='white')

In [ ]:
results_file = 'write/allcells.h5ad'  # the file that will store the analysis results

In [ ]:
#Open the file and reset index of cell and add a column with your condition name.
adata = sc.read_10x_h5('/content/drive/My Drive/RNAseq_folder/sample_filtered_feature_bc_matrix.h5')
adata.obs['your_condition'] = 'your_condition_name'
adata.obs = adata.obs.reset_index()
adata.obs['index'] = adata.obs['index']+'_'+adata.obs['your_condition'].astype(str)
adata.obs = adata.obs.set_index('index')
adata.var_names_make_unique()
adata

In [ ]:
#You can duplicate it for all your samples as below.
adata2 = sc.read_10x_h5('/content/drive/My Drive/RNAseq_folder/2_sample_filtered_feature_bc_matrix.h5')
adata2.obs['your_condition'] = 'your_condition_name_2'
adata2.obs = adata2.obs.reset_index()
adata2.obs['index'] = adata2.obs['index']+'_'+adata2.obs['your_condition'].astype(str)
adata2.obs = adata2.obs.set_index('index')
adata2.var_names_make_unique()
adata2

In [ ]:
#Add more adata objects if needed and concat them.
combined_adata = sc.concat([adata, adata2])
combined_adata

In [ ]:
#Preprocessing.
sc.pl.highest_expr_genes(combined_adata, n_top=20, )

In [ ]:
sc.pp.filter_cells(combined_adata, min_genes=200)
sc.pp.filter_genes(combined_adata, min_cells=3)

In [ ]:
combined_adata.var['mt'] = combined_adata.var_names.str.startswith('MT-')  # annotate the group of mitochondrial genes as 'mt'
sc.pp.calculate_qc_metrics(combined_adata, qc_vars=['mt'], percent_top=None, log1p=False, inplace=True)

In [ ]:
sc.pl.violin(combined_adata, ['n_genes_by_counts', 'total_counts', 'pct_counts_mt'],
             jitter=0.4, multi_panel=True)

In [ ]:
#Remove cells that have too many mitochondrial genes expressed or too many total counts.
sc.pl.scatter(combined_adata, x='total_counts', y='pct_counts_mt')
sc.pl.scatter(combined_adata, x='total_counts', y='n_genes_by_counts')

In [ ]:
#Remove cells that have too many mitochondrial genes expressed or too many total counts.
sc.pl.scatter(combined_adata, x='total_counts', y='pct_counts_mt')
sc.pl.scatter(combined_adata, x='total_counts', y='n_genes_by_counts')

In [ ]:
#Actually do the filtering by slicing the AnnData object.
combined_adata = combined_adata[combined_adata.obs.n_genes_by_counts < 6000, :]
combined_adata = combined_adata[combined_adata.obs.pct_counts_mt < 5, :]

In [ ]:
#Total-count normalize (library-size correct) the data matrix to 10,000 reads per cell, so that counts become comparable among cells. You should change this number based on your data.
sc.pp.normalize_total(combined_adata, target_sum=10000)

In [ ]:
#Logarithmize the data.
sc.pp.log1p(combined_adata)

In [ ]:
#Identify highly-variable genes.
sc.pp.highly_variable_genes(combined_adata, min_mean=0.0125, max_mean=3, min_disp=0.5)

In [ ]:
sc.pl.highly_variable_genes(combined_adata)

In [ ]:
combined_adata.raw = combined_adata

In [ ]:
#Actually do the filtering.
combined_adata = combined_adata[:, combined_adata.var.highly_variable]

In [ ]:
#Regress out effects of total counts per cell and the percentage of mitochondrial genes expressed. Scale the data to unit variance.
sc.pp.regress_out(combined_adata, ['total_counts', 'pct_counts_mt'])

In [ ]:
#Scale each gene to unit variance. Clip values exceeding standard deviation 10.
sc.pp.scale(combined_adata, max_value=10)

In [ ]:
#PCA and computing the neighborhood graph.
sc.pp.pca(combined_adata)
sc.pp.neighbors(combined_adata)
sc.tl.umap(combined_adata)
sc.pl.umap(combined_adata, color='your_condition')

In [ ]:
sc.tl.rank_genes_groups(combined_adata, groupby='your_condition')
sc.pl.rank_genes_groups(combined_adata)

In [ ]:
sc.tl.pca(combined_adata, svd_solver='arpack')

In [ ]:
#Choose a gene name in all capital letters if human gene. Keep first letter in capital and the other not if mouse gene.
sc.pl.pca(combined_adata, color='Actb')

In [ ]:
sc.pl.pca_variance_ratio(combined_adata, log=True)

In [ ]:
combined_adata.write(results_file)
combined_adata

In [ ]:
sc.pp.neighbors(combined_adata, n_neighbors=10, n_pcs=40)

In [ ]:
sc.tl.umap(combined_adata)

In [ ]:
#Choose a gene name in all capital letters if human gene. Keep first letter in capital and the other not if mouse gene.
sc.pl.umap(combined_adata, color=['Cd3e', 'Vim'])

In [ ]:
#Clustering the neighborhood graph.
sc.tl.leiden(combined_adata)

In [ ]:
#Choose a gene name in all capital letters if human gene. Keep first letter in capital and the other not if mouse gene.
sc.pl.umap(combined_adata, color=['leiden','Cd3e'])

In [ ]:
combined_adata.write(results_file)

In [ ]:
sc.settings.verbosity = 2  # reduce the verbosity

In [ ]:
#Finding marker genes and ompute a ranking for the highly differential genes in each cluster.
sc.tl.rank_genes_groups(combined_adata, 'leiden', method='wilcoxon')
sc.pl.rank_genes_groups(combined_adata, n_genes=25, sharey=False)

In [ ]:
combined_adata.write(results_file)

In [ ]:
#Replace with your genes of interest to anotate your clusters.
marker_genes = ['Ptprc','B2m']

In [ ]:
#Reload the object that has been save with the Wilcoxon Rank-Sum test result.
combined_adata = sc.read(results_file)
combined_adata.uns['log1p']["base"] = None

In [ ]:
#Show the 10 top ranked genes per cluster 0, 1, …, 7 in a dataframe.
pd.DataFrame(combined_adata.uns['rank_genes_groups']['names']).head(10)

In [ ]:
#Get a table with the scores and groups.
result = combined_adata.uns['rank_genes_groups']
groups = result['names'].dtype.names
pd.DataFrame(
    {group + '_' + key[:1]: result[key][group]
    for group in groups for key in ['names', 'pvals']}).head(5)

In [ ]:
#If you want to compare a certain gene across groups, use the following.
sc.pl.violin(combined_adata, ['Ptprc','B2m'], groupby="leiden")

In [ ]:
#Actually mark the cell types. Replace with your new cluster names.
new_cluster_names = ["CD4 T","B","Monocytes","NK","CD8 T"]
combined_adata.rename_categories("leiden", new_cluster_names)

In [ ]:
#Save figure.
sc.pl.umap(
    combined_adata, color="leiden", legend_loc="on data", title="", frameon=False, save=".pdf"
)

In [ ]:
#Now that we annotated the cell types, let us visualize the marker genes.
sc.pl.dotplot(combined_adata, marker_genes, groupby='leiden', title='', save='.png');

In [ ]:
combined_adata

In [ ]:
combined_adata.write(results_file, compression='gzip')  # `compression='gzip'` saves disk space, but slows down writing and subsequent reading.

In [ ]:
combined_adata.raw.to_adata().write('./write/allcells_withoutX.h5ad')